# 計算結果をもとにインフォシートを編集する

- https://qiita.com/chiba___/items/c91ba97864efceb81f4d <br>
- https://qiita.com/Chanmoro/items/9a3c86bb465c1cce738a#chrome-%E3%82%92%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB <br>
- https://news.mynavi.jp/techplus/article/zeropython-70/ <br>
- https://news.mynavi.jp/techplus/article/zeropython-66/ <br>
- https://zenn.dev/kazuki_tam/articles/6c3cf0729c5b847cc2a4 <br>
- https://ai-inter1.com/python-selenium/ <br>
- https://regardie.dev/post-3034 <br>
- https://sushiringblog.com/chromedriver-error<br>

- https://gigazine.net/news/20180818-gcp-node-puppeteer-headless-chrome/ <br>
- https://qiita.com/Moh_no/items/e8902ec40df7ddf9448a <br>
- https://blog.crispy-inc.com/entry/2022/02/08/113157 CloudFunctions to Crawring<br>

In [15]:
!pip3 install pycairo
!pip3 install PyPDF2
!pip3 install reportlab
!pip3 install pdfrw
!pip install gspread

  Using cached pycairo-1.23.0.tar.gz (344 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycairo: filename=pycairo-1.23.0-cp37-cp37m-linux_x86_64.whl size=312628 sha256=c268bda6d4d44c372d08f2b55b862a790b99c4232bfc14bbaa61a441d0f61f66
  Stored in directory: /home/jupyter/.cache/pip/wheels/e2/aa/01/80f2cc50a47695378561dfcd8406b09b2ecf8fa66d46177052
Successfully built pycairo


In [18]:
import warnings
import os
import datetime
from tqdm import tqdm
import re
import gspread
import pandas as pd

from pathlib import Path
from PIL import Image
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.lib.units import mm
from pdfrw import PdfReader
from pdfrw.buildxobj import pagexobj
from pdfrw.toreportlab import makerl
from google.cloud import storage
from google.oauth2.service_account import Credentials

warnings.simplefilter('ignore')

In [19]:
dic_unicode={
  'グ':'グ', 'ヴ':'ヴ','ボ':'ボ', 'ヂ':'ヂ', 'ぎ':'ぎ', 'ず':'ず', 'プ':'プ', 'デ':'デ', 'パ':'パ', 'ゼ':'ゼ', 'ぴ':'ぴ', 
  'ぞ':'ぞ', 'ブ':'ブ', 'ギ':'ギ', 'だ':'だ', 'バ':'バ', 'ぽ':'ぽ', 'ズ':'ズ', 'ぷ':'ぷ', 'ポ':'ポ', 'じ':'じ', 'ぢ':'ぢ', 
  'べ':'べ', 'ぱ':'ぱ', 'ジ':'ジ', 'ザ':'ザ', 'び':'び', 'げ':'げ', 'が':'が', 'ビ':'ビ', 'ベ':'ベ', 'ぶ':'ぶ', 'ば':'ば', 
  'ざ':'ざ', 'ペ':'ペ', 'ぼ':'ぼ', 'ヅ':'ヅ', 'ゲ':'ゲ', 'ぺ':'ぺ', 'ガ':'ガ', 'ゴ':'ゴ', 'ゾ':'ゾ', 'ピ':'ピ', 'で':'で', 
  'ぜ':'ぜ', 'ぐ':'ぐ', 'ド':'ド', 'ど':'ど', 'ダ':'ダ', 'づ':'づ', 'ご':'ご','パ':'パ', 'プ':'プ', 'ぴ':'ぴ', 'ポ':'ポ', 
  'ピ':'ピ', 'ぷ':'ぷ', 'ぽ':'ぽ', 'ぱ':'ぱ', 'ぺ':'ぺ', 'ペ':'ペ'
}

In [24]:
# GCSからpdf取得
client=storage.Client("geom-prj-property-recommend")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="geom-prj-property-recommend-70da40021953.json"
bucket=client.get_bucket("info-sheet")
blobs=client.list_blobs(bucket)

##########
# today_date=datetime.datetime.now().strftime('%Y%m%d')
#days=-6 # 今日の日付から遡る日数を、ディレクトリを見て調整
#target_date=(datetime.datetime.now() + datetime.timedelta(days=days)).strftime('%Y%m%d')
target_date = '20230511' # calc_similality.ipynbの実行日に設定する
##########


ls_pdf=[]

for blob in tqdm(blobs):
    if "pdf" in blob.name and blob.name.startswith("全国/"):
        blob_name = blob.name[len("全国/"):]
        ls_pdf.append(blob_name)

        with open(blob_name, "wb") as file_obj:
            blob.download_to_file(file_obj)

                
# SpreadSheet接続設定
secret_credentials_json_oath = 'geom-prj-property-recommend-8d74bd11fa15.json' 
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    secret_credentials_json_oath,
    scopes=scopes
)

gc = gspread.authorize(credentials)

# https://docs.google.com/spreadsheets/d/{ココ}/edit#gid=0
workbook = gc.open_by_key('1WU0WTNOGfXwoWhDXhCPJYZGrF0n4IQLQh-d3Ag4U8n4')
worksheet = workbook.get_worksheet(0)

ls_target=worksheet.get('A1:AI')

593it [01:00,  9.76it/s]


## 関数定義

In [25]:
def cast_unicode(address):
    for uni_word in dic_unicode.keys():
        try:
            address=address.replace(uni_word,dic_unicode[uni_word])
            
        except:
            continue
        
    return address


def make_pdf(brand_name,in_path,file_name,total_score,resident_score,population_score,station_score,retail_score,lon,lat,target_date):
    cc = canvas.Canvas(f"result/{brand_name}/{brand_name}_results_{target_date}/{file_name}.pdf",pagesize=landscape(A4))
    
    # PDFを読み込む
    try:
        pdf = PdfReader(in_path, decompress=False)
    except:
        in_path=cast_unicode(in_path) 
        pdf = PdfReader(in_path, decompress=False)
    
    page = pdf.pages[0]
    pp = pagexobj(page) #ページデータをXobjへの変換
    rl_obj = makerl(cc, pp) # ReportLabオブジェクトへの変換 
    cc.doForm(rl_obj)
    
    # フォント設定
    font_name = "HeiseiKakuGo-W5"
    registerFont(UnicodeCIDFont(font_name))
    cc.setFont(font_name, 13)
    
    # 枠描画
    cc.setFillColorRGB(1,1,1, alpha=1)
    cc.rect(95 * mm, 170 * mm, 100 * mm, 33 * mm, 0, 1)
    
    # 白で塗りつぶす
    cc.setFillColorRGB(0,0,0)
    
    # 文字描画
    height=198
    cc.drawString(93 * mm, height * mm, "ジオマーケティング社独自アルゴリズムに基づく得点")
    
    font_name = "HeiseiKakuGo-W5"
    registerFont(TTFont("GenShinGothic_bold", "GenShinGothic-Bold.ttf"))
    font_bold="GenShinGothic_bold"
    cc.setFont(font_bold, 13)
    
    cc.drawString(93 * mm, (height-7) * mm, "あなたのブランドへの物件レコメンド得点："+str(total_score))
    cc.setFont(font_name, 12)
    cc.drawString(94 * mm, (height-15) * mm, "[あなたのブランドの好調店舗との比較]")
    cc.drawString(94 * mm, (height-20) * mm, "周辺居住者類似度 ："+str(resident_score)+"　商業集積類似度："+str(retail_score))
    cc.drawString(94 * mm, (height-25) * mm, "滞在人口類似度   ："+str(population_score)+"　最寄駅からの距離類似度："+str(station_score))
    
    # 強調するための着色
    cc.setLineWidth(0.8 * mm)
    cc.setStrokeColorRGB(0,191,255)
    cc.rect(92 * mm,  171* mm, 115 * mm, 33 * mm, 1, 0)
    
    # ハイパーリンク（矩形）
    link_address=f'https://app.gleasin.jp/optional_point/result/?lat={lat}&lon={lon}'
    cc.setFillColorRGB(0,191,255)
    cc.rect(170 * mm, (height-15.5) * mm, 14*mm, 4*mm, 0, 1)
    cc.setFillColorRGB(0,0,0)
    cc.drawString(170 * mm, (height-15) * mm, "Check!")
    cc.linkURL(url=link_address, rect=(170 * mm, (height-10) * mm, (170+15)*mm, (height-15)*mm), thickness=1, relative=0)
        
    cc.showPage()
    cc.save()


def make_one_pdf(df,pref,target_date,brand_name):
    for i in tqdm(range(len(df))):
        address=str(df.iloc[i,2] + "_" + df.iloc[i,1])
        
        # FileNotFoundエラーが出たらこの辺いじる
        # address=address.replace("/",":")
        address=address.replace("/","_")
        # address=address.replace(" ","")
        
        
        bld_name=df.iloc[i,1]
        bld_address=str(address+"_"+bld_name)
        
        # 各特徴の点数
        total_score=round(df.iloc[i,3]*100,1)
        resident_score=round(df.iloc[i,4]*100,1)
        population_score=round(df.iloc[i,5]*100,1)
        station_score=round(df.iloc[i,6]*100,1)
        retail_score=round(df.iloc[i,7]*100,1)
                
        lon,lat=df.iloc[i,8],df.iloc[i,9]
       
        make_pdf(brand_name,f'{address}.pdf',f'{pref}_{i+1}',total_score,resident_score,population_score,station_score,retail_score,lon,lat,target_date)

        
def merge_pdf(df,brand_name,company_name,kanji,target_date):
    merger = PyPDF2.PdfMerger()
    for i in tqdm(range(len(df))):
        merger.append(f'result/{brand_name}/{brand_name}_results_{target_date}/{kanji}_{i+1}.pdf')
        
    merger.write(f'result/{brand_name}/{brand_name}_results_{target_date}/{company_name}様_物件レコメンド_{kanji}_{target_date}.pdf')
    merger.close()
    

## Processing

In [26]:
##########
#today_date = datetime.datetime.now().strftime('%Y%m%d')
#days=0 # 今日の日付から遡る日数を、ディレクトリを見て調整
#target_date=(datetime.datetime.now() + datetime.timedelta(days=days)).strftime('%Y%m%d') 
target_date = '20230511' # calc_similality.ipynbの実行日に設定する
##########

for target in ls_target[1:]:
    brand_name = target[33]
    company_name = target[4]
    target_prefs = [p.strip() for p in target[10].split(",")]

    print(brand_name, target_prefs)

    for pref in target_prefs:
        print(pref)
        try:
            df = pd.read_csv(f'result/{brand_name}/{brand_name}_results_{target_date}/df_result_{pref}_{target_date}.csv')
            make_one_pdf(df, pref, target_date, brand_name)
            merge_pdf(df, brand_name, company_name, pref, target_date)
        except:
            print("failure")
            pass

LifeCoach ['大阪府', '東京都', '岐阜県', '愛知県', '京都府', '滋賀県']
大阪府
failure
東京都


100%|██████████| 2/2 [00:00<00:00, 743.87it/s]


岐阜県
failure
愛知県
failure
京都府


100%|██████████| 1/1 [00:00<00:00, 724.28it/s]


滋賀県
failure
fukushin ['埼玉県', '東京都']
埼玉県
failure
東京都


100%|██████████| 5/5 [00:00<00:00, 867.45it/s]

bestbuy ['埼玉県', '千葉県', '東京都', '神奈川県', '岐阜県', '京都府', '大阪府', '兵庫県', '奈良県']
埼玉県
failure
千葉県
failure
東京都
failure
神奈川県
failure
岐阜県
failure
京都府
failure
大阪府
failure
兵庫県
failure
奈良県
failure
tamura ['石川県']
石川県
failure
